<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/youtube/gpu/mandel_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Mandelbrot Plots with an NVIDIA RTX 6000 (Ada)

[Jeff Heaton](https://www.youtube.com/channel/UCR1-GEpyOPzT2AO4D_eifdw)



In [ ]:
import torch
import numpy as np

import PIL.Image
from io import BytesIO
from IPython.display import Image, display

def render(a):
    a_cyclic = (a * 0.3).reshape(list(a.shape) + [1]) #torch.from_numpy(a * 0.3).reshape(list(a.shape) + [1]).to(device)
    img = torch.cat([10 + 20 * torch.cos(a_cyclic),
                     30 + 50 * torch.sin(a_cyclic),
                     155 - 80 * torch.cos(a_cyclic)], 2)
    img[a == a.max()] = 0
    a = img
    a = torch.clamp(a, 0, 255)
    a = a.byte().cpu().numpy()
    f = BytesIO()
    return PIL.Image.fromarray(a)

# Loop through the render cycles for Mandlebrot plot.
def mandelbrot_helper(grid_c, current_values, counts, cycles):
    for i in range(cycles):
        # The Mandlebrot formula
        temp = current_values * current_values + grid_c
        not_diverged = torch.abs(temp) < 4
        current_values.copy_(temp)
        counts.add_(not_diverged.double())

# Render a Mandlebrot plot at the specified location, zoom, and render cycles.
def mandelbrot(render_size, center, zoom, cycles):
    f = zoom / render_size[0]

    real_start = center[1] - (render_size[1] / 2) * f
    real_end = real_start + render_size[1] * f
    imag_start = center[0] - (render_size[0] / 2) * f
    imag_end = imag_start + render_size[0] * f

    real_range = torch.arange(real_start, real_end, f, dtype=torch.float64, device=device)
    imag_range = torch.arange(imag_start, imag_end, f, dtype=torch.float64, device=device)
    real, imag = torch.meshgrid(real_range, imag_range, indexing="ij")
    grid_c = torch.complex(imag, real)
    current_values = torch.clone(grid_c)
    counts = torch.zeros_like(grid_c, dtype=torch.float64, device=device)

    mandelbrot_helper(grid_c, current_values, counts, cycles)
    return counts #counts.cpu().numpy()

# Make use of a GPU or MPS (Apple) if one is available.
device = "mps" if getattr(torch, 'has_mps', False) \
    else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Plot a Single Image

In [ ]:
%%time

counts = mandelbrot(
    #render_size=(30720,17280), # 32K
    #render_size=(15360,8640), # 16K
    #render_size=(7680,4320), # 8K
    #render_size=(3840,2160), # 4K
    render_size=(1920,1080), # HD
    center=(-0.5,0),
    zoom=4,
    cycles=200
)  
img = render(counts)
img.save("render.png")
print(img.size)
img

# Find Zoom Parameters

To perform a mandlebrot Zoom you must find an interesting spot to zoom in on. To do this start with a zoom of 1, and center of 0,0. Increase your zoom and continue to modify the center so that something interesting stays in frame as you increase the zoom.

In [ ]:
zoom_amount = 3000
center = (-0.701000102001,0.35)

zoom = 0.99 ** zoom_amount
counts = mandelbrot(
    #render_size=(30720,17280), # 32K
    #render_size=(15360,8640), # 16K
    #render_size=(7680,4320), # 8K
    #render_size=(3840,2160), # 4K
    render_size=(1920,1080), # HD
    center=center,
    zoom=zoom,
    cycles=300
)  
img = render(counts)
print(img.size)
img

# Render a Zoom Video

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!mkdir tmp

In [ ]:
from tqdm import tqdm

zoom = 1.0
for i in tqdm(range(3000)):
  counts = mandelbrot(
      (3840,2160),
      #(640,480),
      (-0.701000102001,0.35),
      zoom,
      2000
  )
  zoom *=0.99
  img = render(counts)
  img.save(f"./tmp/img-{i}.png")

print(zoom)

Package the frames to a MP4.

In [ ]:
!ffmpeg -r 30 -i ./tmp/img-%d.png -vcodec mpeg4 -b 5000k -y movie.mp4

Download the movie, if you are using colab.

In [ ]:
from google.colab import files
files.download('movie.mp4') 
#files.download('/content/tmp/img-300.png')

# -vcodec libx264